<a href="https://colab.research.google.com/github/FranciscoFoz/Projeto_Final_Bootcamp_Data_Science_Alura/blob/main/Notebooks/Notebook_de_Funcoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color= MediumSpringGreen		size = 6> Bootcamp Data Science Alura - Projeto Final </font>

# <font color= MediumSpringGreen		size = 6> Notebook de funções

## <font color= MediumSpringGreen> 1. Funções de manipulação de dados


### <font color= MediumSpringGreen> 1.1 remover_pacientes_com_primeira_janela_positiva_para_UTI

Esta função irá remover os pacientes que possuam resultado positivo(1) para entrada na UTI dentro da primeira janela (0-2)

In [ ]:
def remover_pacientes_com_primeira_janela_positiva_para_UTI(dados):
  a_remover = dados.query("WINDOW == '0-2' and ICU == 1")['PATIENT_VISIT_IDENTIFIER'].values            #Query dos pacientes que possuam a primeira janela e resultado positivo
  dados_limpos = dados.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")                              #Query removendo os pacientes na condição acima
  return dados_limpos

### <font color= MediumSpringGreen> 1.2 preenche_tabela
Esta função irá preencher a tabela onde esteja NAN com os valores anteriores e depois com os posteriores.

In [3]:
def preenche_tabela(dados):
  features_continuas_colunas = dados.iloc[:,4:-2].columns                                                                                                   #Selecionar colunas de variáveis contínuas
  features_continuas = dados.groupby('PATIENT_VISIT_IDENTIFIER', as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')  #Preencher os dados NAN da tabela
  features_categoricas = dados.iloc[:,:4]                                                                                                                   #Selecionar as categóricas
  saida = dados.iloc[:, -2:]                                                                                                                                #Selecionar os dados de saída
  dados_finais = pd.concat([features_categoricas,features_continuas,saida],ignore_index=True,axis=1)                                                        #Concatenar os dados anteriores
  dados_finais.columns = dados.columns                                                                                                                      #Renomear as colunas
  return dados_finais

### <font color= MediumSpringGreen> 1.2 prepare_window
Esta função irá localizar todas as linhas que possuam janela entre "0-2" e UTI igual a 1, retornando as linhas.

In [8]:
def prepare_window(rows):                                                   
  if(np.any(rows['ICU'])):                                                      
    rows.loc[rows['WINDOW']=='0-2','ICU'] = 1                                   #Condição se para localizar as linhas com os filtros desejados.
  return rows.loc[rows['WINDOW'] == '0-2']

### <font color= MediumSpringGreen> 1.3 excluir_coluna_id_paciente
Esta função irá excluir a coluna do ID do paciente.

In [1]:
def excluir_coluna_id_paciente(dados):
  dados_primeira_janela.drop(['PATIENT_VISIT_IDENTIFIER'],axis=1)               #Excluir coluna do id do paciente

### <font color= MediumSpringGreen> 1.4 transformar_AGE_PERCENTIL_em_dados_categoricos
Esta função irá excluir a coluna do ID do paciente.

In [2]:
def transformar_AGE_PERCENTIL_em_dados_categoricos(dados):
  dados.AGE_PERCENTIL.astype('category').cat.codes                              #Transformar coluna AGE PECENTIL em dados categóricos

### <font color= MediumSpringGreen> 1.5 remover_variaveis_correlacionadas
Esta função irá filtrar os dados entre as variáveis, excluindo: "AGE_ABOVE65","AGE_PERCENTIL", "GENDER", "WINDOW" e "ICU".

Criará uma matriz de correlação entre eles, permanecendo apenas com o triângulo superior.

Excluirá as colunas com correção passada como parâmetro.


In [3]:
def remover_variaveis_correlacionadas(dados,valor_de_corte):

  matriz_corr = dados_limpos.iloc[:,3:-2].corr().abs()                                                        #Filtrar dados correlacionando-os e transformando-os em números absolutos.
  matriz_superior = matrix_corr.where(np.triu(np.ones(matriz_corr.shape),k=1).astype(np.bool))                #Criar matriz superior da matriz de correlação
  excluir = [coluna for coluna in  matriz_superior if any(matriz_superior[coluna] > valor_de_corte)]          #Selecionar colunas dentro da matriz superior que contenham alta correlação

  return dados.drop(excluir,axis=1)                                                                           #Retornar o dataframe sem as colunas com alta correlação